In [1]:
# Dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Setting up splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
INFO:WDM:Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
INFO:WDM:Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\Ray\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache
INFO:WDM:Driver [C:\Users\Ray\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


In [3]:
# URLs to scrape
url_news = 'https://redplanetscience.com/'
url_imgs = 'https://spaceimages-mars.com'
url_info = 'https://galaxyfacts-mars.com'
url_hemi = 'https://marshemispheres.com/'

In [4]:
# Working with redplanetscience
browser.visit(url_news)

In [5]:
# HTML object
html_news = browser.html

# Using Beautiful Soup
soup_news = bs(html_news,'html.parser')

# Identifying elements with needed information
article = soup_news.find('div', class_='list_text')

In [6]:
# Saving information to variables
news_title = article.find('div', class_='content_title').text
print(news_title)

news_text = article.find('div', class_='article_teaser_body').text
print(news_text)

NASA Wins Two Emmy Awards for Interactive Mission Coverage
NASA-JPL's coverage of the Mars InSight landing earns one of the two wins, making this the NASA center's second Emmy.


In [7]:
# Working with spaceimages
browser.visit(url_imgs)

In [8]:
# HTML Object
html_imgs = browser.html

# Using Beautiful Soup
soup_imgs = bs(html_imgs, 'html.parser')

# Identifying elements with needed information
image = soup_imgs.find('img', class_='headerimage fade-in')

In [9]:
# Creating url for featured image, saving to variable
featured_image_url = url_imgs + '/' + image['src']
print(featured_image_url)

https://spaceimages-mars.com/image/featured/mars3.jpg


In [10]:
# Scraping tablular data with pandas
html_info = pd.read_html(url_info)
#html_info

In [11]:
# Saving html string to variable
table_str = html_info[0].to_html(classes='table')
#print(table_str)

In [12]:
# Working with marshemispheres
browser.visit(url_hemi)

In [13]:
# HTML Object
html_hemi = browser.html

# Using Beautiful Soup
soup_hemi = bs(html_hemi, 'html.parser')

# Identifying elements with needed information
hemispheres = soup_hemi.find_all('div', class_='description')
#print(hemispheres)

In [14]:
# Empty lists for iteration
hem_title = []
hemisphere_pages = []

# Iterate through list of results to find image data
for h in hemispheres:
    
    # Save name of image
    img_name = h.find('h3')
    hem_name = img_name.text
    hem_title.append(hem_name)

    # Save urls of each hemisphere page, to visit later for full images
    hemlink = h.find('a')
    hemurl = url_hemi + hemlink['href']
    hemisphere_pages.append(hemurl)

hem_title

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

In [15]:
# Empty list for iteration
hem_img_urls = []

# Iterate through list of urls to find full images
for p in hemisphere_pages:

    # Visit url specified
    browser.visit(p)

    # Setting up html and soup
    current_html = browser.html
    current_soup = bs(current_html, 'html.parser')

    # Find object with image, extract its link
    hem_img = current_soup.find('img', class_='wide-image')
    hem_img_url = url_hemi + hem_img['src']

    # Save image url
    hem_img_urls.append(hem_img_url)

hem_img_urls

['https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg']

In [16]:
# Empty list for iteration
hem = []

# Iterate through lists of titles and urls, with 4 items each
for i in range(4):

    # Save name and image url as a key:value pair
    hem_dict = {'title': hem_title[i - 1], 'img_url': hem_img_urls[i - 1]}
    hem.append(hem_dict)

hem

[{'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'},
 {'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}]

In [17]:
# Exiting browser
browser.quit()